In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import math

In [2]:
export_dir = './final_model/'
batch_size = 10000
disp_log = 50

In [3]:
# write predict to file
dw = pd.read_csv('./DL_final_project/DL_Taiwan_data/sinica/201703_Taiwan.csv')
dfw = dw.assign(Timestamp = pd.to_datetime(dw['Date']+' '+dw['Time']))
dfw = dfw.assign(Hour = dfw['Timestamp'].dt.hour)
dfw = dfw[['Hour','PM10','PM1','Temperature','Humidity','lon','lat']]

predict_values = np.zeros(len(dfw))
n_batches = math.ceil(len(dfw) / batch_size)
print('Total batches:',n_batches)

Total batches: 420


In [5]:
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, ["tag"], export_dir)
    graph = tf.get_default_graph()
    X = graph.get_tensor_by_name("Input/X:0")
    is_training = graph.get_tensor_by_name("Input/is_training:0")
    predict = graph.get_tensor_by_name("Predict/predict:0")
    
    for i in range(n_batches):
        if i % disp_log == 0:
            print('batch',i)
        if i == n_batches - 1:
            X_batch = dfw[i*batch_size:].values
            predict_batch = sess.run(predict, {is_training: False, X: X_batch})
            predict_values[i*batch_size:] = predict_batch[:,0]
        else:
            X_batch = dfw[i*batch_size:(i+1)*batch_size].values
            predict_batch = sess.run(predict, {is_training: False, X: X_batch})
            predict_values[i*batch_size:(i+1)*batch_size] = predict_batch[:,0]
        
    # Fixme!!
    print("predicted_values len =",len(predict_values))
    predicted_df = pd.DataFrame({'PM2.5':predict_values})
    dw['PM2.5']=predicted_df['PM2.5']
    print('saving to ./predict.csv')
    dw.to_csv('./predict.csv', index=False)
    print('saved to ./predict.csv')

INFO:tensorflow:Restoring parameters from b'./final_model/variables\\variables'
batch 0
batch 50
batch 100
batch 150
batch 200
batch 250
batch 300
batch 350
batch 400
predicted_values len = 4191423
saving to ./predict.csv
saved to ./predict.csv
